# Ski regressor
## Prices may vary

In [45]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error

df = pd.read_csv('Shiny_data1.csv', index_col=0)
df = df.drop(df[df.ski_pass_price == 0].index)

## Skiing is expensive.
### It only really starts to sink in when you have to pay for it yourself. 
### I've scraped a bit of ski resort data from ski-resort-stats.com to help me make more informed decisions. Let's see if we can build a model to predict the ski pass price and make better decisions about where to plan our next holiday.

In [2]:
display(df.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 501 entries, 0 to 514
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   child_friendly       501 non-null    bool   
 1   continent            501 non-null    object 
 2   country              501 non-null    object 
 3   max_altitude         501 non-null    float64
 4   min_altitude         501 non-null    float64
 5   resort_name          501 non-null    object 
 6   season               501 non-null    object 
 7   ski_pass_price       501 non-null    int64  
 8   url                  501 non-null    object 
 9   beginner_slopes      138 non-null    float64
 10  intermediate_slopes  137 non-null    float64
 11  difficult_slopes     136 non-null    float64
 12  t-bar_lifts          138 non-null    float64
 13  chairlifts           138 non-null    float64
 14  gondolas             136 non-null    float64
 15  snowpark             142 non-null    obj

None

A lot of collumns of different lengths! 
We'll stick with the full length collumns as our data is pretty sparse as it is.

In [53]:
df.drop(['country'], axis = 1, inplace=True)
df.drop(['resort_name'], axis = 1, inplace=True)
df.drop(df.iloc[:, 6:16], axis = 1, inplace=True)
df.drop(df.iloc[:,9:11], axis = 1, inplace=True)
print(df.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 501 entries, 0 to 514
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   child_friendly  501 non-null    bool   
 1   continent       501 non-null    object 
 2   max_altitude    501 non-null    float64
 3   min_altitude    501 non-null    float64
 4   season          501 non-null    object 
 5   ski_pass_price  501 non-null    int64  
 6   country_iso     501 non-null    object 
 7   altitude_diff   501 non-null    float64
 8   total_slopes    501 non-null    float64
dtypes: bool(1), float64(4), int64(1), object(3)
memory usage: 29.8+ KB
None


Much better, I suspect we'll get some colinearity problems from altitude-diff and min,max altitude but we'll start with these and see how the model performs.
Let's do some exploratory data analysis.

In [71]:
px.imshow(df.corr(), color_continuous_scale='Agsunset', title="Correlation heatmap of Skidata")

There seems to be a pretty high correlation between min and max altitude as well as between max altitude and the altitude difference which makes sense. My main intuition is that there's a significant relationship between the ski pass price and the altitude difference. 

In [60]:
fig = px.scatter(df,x='altitude_diff', y='ski_pass_price')
fig.show()

The scatter plot shows a correlation between the ski pass price and altitude difference, however there seems to be two distinct distributions. I suspect it's because of disparities in price between Europe and America. Let's plot them seperately and see.

In [85]:
euro = df[df['continent'] == 'Europe'].copy()
america = df[df['continent'] == 'America'].copy()
rest = df[df['continent'] == 'Rest of the world'].copy()

In [61]:
fig = make_subplots(
    rows=2,cols=2,
    subplot_titles=('All', 'Europe', 'America', 'Rest of the World'))

fig.add_trace(go.Scatter(x=df['altitude_diff'], y=df['ski_pass_price'], mode='markers'),row=1,col=1)
fig.add_trace(go.Scatter(x=euro['altitude_diff'], y=euro['ski_pass_price'], mode='markers'),row=1,col=2)
fig.add_trace(go.Scatter(x=america['altitude_diff'], y=america['ski_pass_price'], mode='markers'),row=2,col=1)
fig.add_trace(go.Scatter(x=rest['altitude_diff'], y=rest['ski_pass_price'], mode='markers'),row=2,col=2)

As suspected there seems to be distinct distributions for each 'continent'. This should hopefully be captured by that variable in the regression model. Let's split the data and start modelling!

In [94]:
train, Test = train_test_split(df, train_size = 0.6)
Test, Validate = train_test_split(Test, train_size = 0.5)

In [95]:
f = 'ski_pass_price ~ ' + ' + '.join(df.columns.drop('ski_pass_price'))
model = smf.ols(formula = f, data = train).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         ski_pass_price   R-squared:                       0.792
Model:                            OLS   Adj. R-squared:                  0.737
Method:                 Least Squares   F-statistic:                     14.27
Date:                Sat, 10 Apr 2021   Prob (F-statistic):           1.28e-52
Time:                        19:58:27   Log-Likelihood:                -1109.1
No. Observations:                 300   AIC:                             2346.
Df Residuals:                     236   BIC:                             2583.
Df Model:                          63                                         
Covariance Type:            nonrobust                                         
=========================================================================================================================================
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                -7.6482      8.554     -0.894      0.372     -24.500       9.204
child_friendly[T.True]                                                   37.0585      9.137      4.056      0.000      19.059      55.058
continent[T.Europe]                                                     -11.3753      5.990     -1.899      0.059     -23.177       0.426
continent[T.Rest of the world]                                            2.1647      3.611      0.599      0.549      -4.950       9.279
season[T.December - April June - August October - November]              -5.8234     11.309     -0.515      0.607     -28.103      16.456
season[T.December - March]                                               -3.3196      2.703     -1.228      0.221      -8.644       2.005
season[T.December - May]                                                 -7.3801      7.583     -0.973      0.331     -22.319       7.559
season[T.December - depending on snow conditions]                         2.9913     11.203      0.267      0.790     -19.079      25.062
season[T.July - April]                                                   -0.1138     11.307     -0.010      0.992     -22.389      22.161
season[T.July - September]                                              -18.9057     12.199     -1.550      0.123     -42.939       5.128
season[T.June - May]                                                      5.8551     11.154      0.525      0.600     -16.119      27.829
season[T.June - October]                                                 22.5528     10.813      2.086      0.038       1.250      43.856
season[T.June - September]                                               15.3426      6.686      2.295      0.023       2.172      28.513
season[T.May - September]                                                 0.0799     12.166      0.007      0.995     -23.889      24.049
season[T.November - April]                                                4.5782      2.221      2.061      0.040       0.202       8.955
season[T.November - June]                                                28.5943     11.326      2.525      0.012       6.281      50.908
season[T.November - March]                                               -0.7917     11.242     -0.070      0.944     -22.940      21.356
season[T.November - May]                                                  2.0391      2.963      0.688      0.492      -3.799       7.877
season[T.November - May June - August]                                   12.7283      8.229      1.547      0.123      -3.482      28.939
season[T.October - April]                      

There are a lot of predictors in this model especially for the country and season variables. We get a high R-squared but I suspect it won't generalise very well.

In [96]:
predictions = model.predict(Test)

PatsyError: predict requires that you use a DataFrame when predicting from a model
that was created using the formula api.

The original error message returned by patsy is:
Error converting data to categorical: observation with value 'October - November December - May June - October' does not match any of the expected levels (expected: ['December - April', 'December - April June - August October - November', ..., 'depending on snow conditions - depending on snow conditions', 'no report'])
    ski_pass_price ~ child_friendly + continent + max_altitude + min_altitude + season + country_iso + altitude_diff + total_slopes
                                                                                ^^^^^^

There's too many categories and not enough data! Looks like we need to drop Season and Country as predictors.

In [97]:
f = 'ski_pass_price ~ ' + ' + '.join(df.columns.drop(['ski_pass_price','season','country_iso']))
model = smf.ols(formula = f, data = train).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         ski_pass_price   R-squared:                       0.589
Model:                            OLS   Adj. R-squared:                  0.580
Method:                 Least Squares   F-statistic:                     69.86
Date:                Sat, 10 Apr 2021   Prob (F-statistic):           1.19e-53
Time:                        19:58:38   Log-Likelihood:                -1211.5
No. Observations:                 300   AIC:                             2437.
Df Residuals:                     293   BIC:                             2463.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                         57.1462      7.663      7.457      0.000      42.065      72.228
child_friendly[T.True]             3.4937      7.206      0.485      0.628     -10.687      17.675
continent[T.Europe]              -35.7531      2.157    -16.578      0.000     -39.998     -31.509
continent[T.Rest of the world]   -25.7956      3.765     -6.852      0.000     -33.205     -18.386
max_altitude                       0.0049      0.001      6.463      0.000       0.003       0.006
min_altitude                      -0.0002      0.001     -0.173      0.863      -0.003       0.002
altitude_diff                      0.0051      0.001      3.661      0.000       0.002       0.008
total_slopes                       0.0323      0.009      3.568      0.000       0.015       0.050
==============================================================================
Omnibus:                       18.935   Durbin-Watson:                   2.062
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               42.449
Skew:                           0.278   Prob(JB):                     6.06e-10
Kurtosis:                       4.757   Cond. No.                     6.47e+15
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.75e-23. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

Rsquared has gone down as expected and min_altitude is not significant at a=0.05 so we'll drop it.

In [98]:
f = 'ski_pass_price ~ ' + ' + '.join(df.columns.drop(['ski_pass_price','season','country_iso', 'min_altitude']))
model = smf.ols(formula = f, data = df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         ski_pass_price   R-squared:                       0.577
Model:                            OLS   Adj. R-squared:                  0.572
Method:                 Least Squares   F-statistic:                     112.2
Date:                Sat, 10 Apr 2021   Prob (F-statistic):           6.02e-89
Time:                        19:58:43   Log-Likelihood:                -2017.6
No. Observations:                 501   AIC:                             4049.
Df Residuals:                     494   BIC:                             4079.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                         56.4400      7.285      7.748      0.000      42.127      70.753
child_friendly[T.True]             4.0362      6.991      0.577      0.564      -9.700      17.772
continent[T.Europe]              -34.9398      1.685    -20.733      0.000     -38.251     -31.629
continent[T.Rest of the world]   -23.9408      2.966     -8.070      0.000     -29.769     -18.112
max_altitude                       0.0042      0.001      3.793      0.000       0.002       0.006
altitude_diff                      0.0060      0.002      3.212      0.001       0.002       0.010
total_slopes                       0.0309      0.007      4.336      0.000       0.017       0.045
==============================================================================
Omnibus:                       42.871   Durbin-Watson:                   1.630
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               99.936
Skew:                           0.457   Prob(JB):                     1.99e-22
Kurtosis:                       4.988   Cond. No.                     4.08e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.08e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Rsquared has gone up! But child_friendly is no longer significant so let's drop it and continue.

In [103]:
f = 'ski_pass_price ~ ' + ' + '.join(df.columns.drop(['ski_pass_price','season','country_iso', 'child_friendly', 'min_altitude']))
model = smf.ols(formula = f, data = train).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         ski_pass_price   R-squared:                       0.588
Model:                            OLS   Adj. R-squared:                  0.581
Method:                 Least Squares   F-statistic:                     84.00
Date:                Sat, 10 Apr 2021   Prob (F-statistic):           1.39e-54
Time:                        20:01:26   Log-Likelihood:                -1211.6
No. Observations:                 300   AIC:                             2435.
Df Residuals:                     294   BIC:                             2458.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                         60.5356      3.135     19.309      0.000      54.366      66.706
continent[T.Europe]              -35.7449      2.154    -16.596      0.000     -39.984     -31.506
continent[T.Rest of the world]   -26.1228      3.699     -7.062      0.000     -33.403     -18.843
max_altitude                       0.0046      0.001      3.181      0.002       0.002       0.007
altitude_diff                      0.0054      0.002      2.187      0.030       0.001       0.010
total_slopes                       0.0322      0.009      3.560      0.000       0.014       0.050
==============================================================================
Omnibus:                       19.637   Durbin-Watson:                   2.060
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               47.954
Skew:                           0.255   Prob(JB):                     3.86e-11
Kurtosis:                       4.891   Cond. No.                     1.36e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.36e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

All the variables are significant so let's see how she performs.

In [104]:
def plot_scatter_and_line(x, scatter_y, line_y, scatter_name, line_name, title, x_title, y_title):

    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=x, y=scatter_y, name=scatter_name, mode="markers"))
    fig.add_trace(go.Scatter(
        x=x, y=line_y, name=line_name))
    fig.update_layout(title=title, xaxis_title=x_title,
        yaxis_title=y_title)
    
    return fig

In [105]:
Test['predictions'] = model.predict(Test)
px.scatter(Test, x= 'predictions', y = 'ski_pass_price')
line_y = Test['ski_pass_price']
plot_scatter_and_line(Test['ski_pass_price'], Test['predictions'], line_y, 'Predictions', 'y=x','Predicted vs true price', 'Pass_px', 'Predicted price')


Seems to be some outliers and quite a bit of variance. Let's calculate the RMSE.

In [106]:
RMSE = np.sqrt(mean_squared_error(Test['predictions'],Test['ski_pass_price']))
mean_pass = Test['ski_pass_price'].mean()
print('RMSE: ', RMSE)
print()
print('Mean ski pass price: ', mean_pass)
print()
print('Comparison: ', RMSE/mean_pass *100, '%')


RMSE:  12.762805604898878

Mean ski pass price:  50.37

Comparison:  25.338109201705137 %


RMSE is about 25% of the mean price... Not a brilliant model. Let's investigate by plotting the residuals of the model.

In [108]:
line_y = [0] * len(train['ski_pass_price'])
plot_scatter_and_line(train['ski_pass_price'], model.resid, line_y, 'Model residuals', 'y=0','Model residual plot', 'Pass_px', 'Residuals')


Seems like the model didn't catch the two distinct distributions and left a lot of predictability by the wayside! The residuals are pretty linear so the model doesn't seem to be capturing the variance of the data very well. Let's see if we can do better by only modelling the european data.

In [86]:
trainEU, TestEU = train_test_split(euro, train_size = 0.6)
TestEU, ValidateEU = train_test_split(TestEU, train_size = 0.5)

euro_mod = smf.ols(formula= f, data = trainEU).fit()
euro_mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         ski_pass_price   R-squared:                       0.394
Model:                            OLS   Adj. R-squared:                  0.385
Method:                 Least Squares   F-statistic:                     46.77
Date:                Sat, 10 Apr 2021   Prob (F-statistic):           2.49e-23
Time:                        18:41:12   Log-Likelihood:                -777.96
No. Observations:                 220   AIC:                             1564.
Df Residuals:                     216   BIC:                             1577.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        30.6100      1.777     17.227      0.000      27.108      34.112
max_altitude     -0.0010      0.001     -0.677      0.499      -0.004       0.002
altitude_diff     0.0121      0.002      5.896      0.000       0.008       0.016
total_slopes      0.0206      0.006      3.210      0.002       0.008       0.033
==============================================================================
Omnibus:                        3.383   Durbin-Watson:                   1.791
Prob(Omnibus):                  0.184   Jarque-Bera (JB):                3.099
Skew:                          -0.286   Prob(JB):                        0.212
Kurtosis:                       3.109   Cond. No.                     7.72e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.72e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [89]:
TestEU['predictions'] = euro_mod.predict(TestEU)
px.scatter(TestEU, x= 'predictions', y = 'ski_pass_price')
line_y = TestEU['ski_pass_price']
plot_scatter_and_line(TestEU['ski_pass_price'], TestEU['predictions'], line_y, 'Predictions', 'y=x','Predicted vs true price', 'Pass_px', 'Predicted price')

Seems a bit flatter than it should be. Let's see what the residuals look like.

In [91]:
line_y = [0] * len(TestEU['ski_pass_price'])
plot_scatter_and_line(TestEU['ski_pass_price'], euro_mod.resid, line_y, 'Model residuals', 'y=0','Euro Model residual plot', 'Pass_px', 'Residuals')


Residuals are a lot better, looking independent although homoescadasicity not the best, probably because of the small training set.

In [92]:
RMSE = np.sqrt(mean_squared_error(TestEU['predictions'],TestEU['ski_pass_price']))
mean_pass = TestEU['ski_pass_price'].mean()
print('RMSE: ', RMSE)
print()
print('Mean ski pass price: ', mean_pass)
print()
print('Comparison: ', RMSE/mean_pass *100, '%')

RMSE:  7.997002780733124

Mean ski pass price:  40.25675675675676

Comparison:  19.864995158585135 %


RMSE is a lot better than the previous model, we've gone down to about 20% of the mean price. Let's compare our two models on the validation data sets.

In [113]:
Validate['predictions'] = model.predict(Validate)
ValidateEU['predictions'] = model.predict(ValidateEU)

RMSE_all = np.sqrt(mean_squared_error(Validate['predictions'], Validate['ski_pass_price']))
mean_passAll = Validate['ski_pass_price'].mean()
RMSE_EU = np.sqrt(mean_squared_error(ValidateEU['predictions'], ValidateEU['ski_pass_price']))
mean_passEU = ValidateEU['ski_pass_price'].mean()

In [114]:
print('RMSE: ', RMSE_all)
print('Mean ski pass price: ', mean_passAll)
print('Comparison: ', RMSE_all/mean_passAll *100, '%')

RMSE:  13.974064217198942
Mean ski pass price:  47.495049504950494
Comparison:  29.42214896679369 %


In [115]:
print('RMSE: ', RMSE_EU)
print('Mean ski pass price: ', mean_passEU)
print('Comparison: ', RMSE_EU/mean_passEU *100, '%')

RMSE:  8.87210927160194
Mean ski pass price:  43.63513513513514
Comparison:  20.332489504445448 %


The general model doesn't seem to generalise very well, RMSE increases for the validation set. For the 'eurocentric' model RMSE remains about the same so seems to be the better of the two.

All in all we don't seem to have enough data to make a model that produces decent predictions. However sometimes good enough is better than not at all. 